In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.layers import Attention
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
import pandas as pd
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import shapiro
import statsmodels.api as sm
from numpy.lib.stride_tricks import as_strided
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

from keras.models import Sequential, Model
from keras.layers import Conv1D, AveragePooling1D, Flatten, Activation, Conv2D, BatchNormalization, Input, Concatenate, Dense, MaxPooling1D, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [23]:
file_path = 'data_extracted/1.csv'
data = pd.read_csv(file_path, header=None)

df = np.array_split(data, 100)
print(df.shape)

y = df[2].astype(int)
X = df[1]

print(X.shape)
print(y.shape)

c:\Users\egrka\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


AttributeError: 'list' object has no attribute 'shape'

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, TimeDistributed, Bidirectional

def create_model():
    # Define the encoder
    encoder_inputs = Input(shape=(None, 1))
    encoder = Bidirectional(LSTM(64, return_sequences=True))(encoder_inputs)
    encoder = Dropout(0.2)(encoder)
    encoder_outputs, state_h, state_c = LSTM(64, return_state=True)(encoder)
    encoder_states = [state_h, state_c]

    # Define the decoder
    decoder_inputs = Input(shape=(None, 3))  # 3 classes: 0, 1, 2
    decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_outputs = Dropout(0.2)(decoder_outputs)
    decoder_dense = TimeDistributed(Dense(3, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [21]:
encoder_inputs = np.expand_dims(X_train, axis=0)  # Shape: (1, 50)
encoder_input_data = np.expand_dims(encoder_inputs, axis=-1) # Shape: (1, 50, 1)

start_token = 0  # Start token
decoder_inputs = np.insert(y_train[:-1], 0, start_token)  # Shift and add start token

num_classes = 3
decoder_input_data = np.eye(num_classes)[decoder_inputs]  # 50, 3)
decoder_input_data = np.expand_dims(decoder_input_data, axis=0)  # Shape: (1, 50, 3)

decoder_targets = np.eye(num_classes)[y_train]  # (batch_size=100, sequence_length=50, 3)
decoder_target_data = np.expand_dims(decoder_targets, axis=0)  # Shape: (1, 50, 3)

print("Encoder input shape:", encoder_input_data.shape)  # Should be (batch_size, sequence_length, 1)
print("Decoder input shape:", decoder_input_data.shape)  # Should be (batch_size, sequence_length, 3)
print("Decoder target shape:", decoder_target_data.shape)  # Should be (batch_size, sequence_length, 3)

Encoder input shape: (1, 1040, 3, 1)
Decoder input shape: (1, 3118, 3)
Decoder target shape: (1, 1040, 3, 3)


In [6]:
encoder_input_data

array([[[-0.34911427],
        [-0.24434906],
        [-0.1484023 ],
        ...,
        [ 0.08298283],
        [ 0.02861094],
        [-0.02501427]]])

In [7]:
# model = create_model()

# model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=1, epochs=50)

In [15]:
# K-Fold Cross-Validation
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Store results
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"Training Fold {fold + 1}/{k}")
    print(train_idx, val_idx)

    # Split data into training and validation sets
    encoder_train, encoder_val = X_train[train_idx], X_train[val_idx]
    decoder_input_train, decoder_input_val = decoder_inputs[train_idx], decoder_inputs[val_idx]
    decoder_target_train, decoder_target_val = decoder_targets[train_idx], decoder_targets[val_idx]

    # Create batches for training and validation sets
    batch_size = 32

    # Training data
    train_data = tf.data.Dataset.from_tensor_slices(
        ([encoder_train, decoder_input_train], decoder_target_train)
    ).shuffle(len(train_idx)).batch(batch_size)

    # Validation data
    val_data = tf.data.Dataset.from_tensor_slices(
        ([encoder_val, decoder_input_val], decoder_target_val)
    ).batch(batch_size)

    # Create and train the model
    model = create_model()
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=10,
        verbose=1
    )

    # Store validation results
    val_loss, val_accuracy = model.evaluate(val_data, verbose=0)
    results.append((val_loss, val_accuracy))
    print(f"Fold {fold + 1} - Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

# Average results across folds
avg_val_loss = np.mean([result[0] for result in results])
avg_val_accuracy = np.mean([result[1] for result in results])
print(f"\nAverage Validation Loss: {avg_val_loss}")
print(f"Average Validation Accuracy: {avg_val_accuracy}")

Training Fold 1/5
[     0      1      2 ... 103995 103996 103997] [     8     23     24 ... 103991 103992 103998]


ValueError: Dimensions 2 and 83199 are not compatible